In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
dataset_path = '../input/felidae-tiger-lion-cheetah-leopard-puma/Felidae'

In [3]:
from pathlib import Path
from collections import Counter

dataset_path = Path(dataset_path)

In [4]:
paths = list(dataset_path.glob('*/*'))
classes = dict(Counter([path.parent.stem for path in paths]))
extensions = dict(Counter([path.suffix for path in paths]))
classes, extensions

({'Cheetah': 56, 'Lion': 49, 'Tiger': 48, 'Puma': 40, 'Leopard': 50},
 {'.jpg': 243})

In [5]:
# check dataset for non-jpg even though they have .jpg extension files:
from PIL import Image

formats = []
for path in paths:
    im = Image.open(path)
    formats.append(im.format)
    im.close()
    
Counter(formats)

Counter({'JPEG': 180, 'WEBP': 61, 'PNG': 2})

In [6]:
# tensorflow cannot handle webp images
# converting webp and png to jpg

new_ds_path = Path('./NewDataset')
new_ds_path.mkdir(parents=True, exist_ok=True)
for c in classes:
    class_path = new_ds_path / c
    class_path.mkdir(parents=True, exist_ok=True)

In [7]:
!tree

.
├── NewDataset
│   ├── Cheetah
│   ├── Leopard
│   ├── Lion
│   ├── Puma
│   └── Tiger
└── __notebook__.ipynb

6 directories, 1 file


In [8]:
for path in paths:
    im = Image.open(path)
    im = im.convert(mode='RGB')
    class_name = path.parent.stem
    file_name = path.stem
    fp = new_ds_path.joinpath(class_name).joinpath(file_name)
    fp = fp.parent / (fp.name + '.jpg')
#     print(fp)
    im.save(fp, format='JPEG')

In [9]:
class Config:
    batch_size = 16
    img_dim = (224,224)
    epochs = 20

In [10]:
dataset_path = Path('./NewDataset')

In [11]:
train_ds = keras.utils.image_dataset_from_directory(dataset_path,
                                                    subset = 'training',
                                                    validation_split = 0.2,
                                                    image_size = Config.img_dim,
                                                    batch_size = Config.batch_size,
                                                    seed=1
                                                   )

val_ds = keras.utils.image_dataset_from_directory(dataset_path,
                                                  subset = 'validation',
                                                  validation_split = 0.2,
                                                  image_size = Config.img_dim, 
                                                  batch_size = Config.batch_size,
                                                  seed=1
                                                 )

Found 243 files belonging to 5 classes.
Using 195 files for training.


2022-07-06 07:30:04.055735: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 07:30:04.203464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 07:30:04.204467: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 07:30:04.207814: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Found 243 files belonging to 5 classes.
Using 48 files for validation.


node zero
2022-07-06 07:30:04.210819: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 07:30:04.211927: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 07:30:07.058064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 07:30:07.058994: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 07:30:07.059746: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node r

In [12]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal", input_shape=(*Config.img_dim, 3)),
    layers.RandomContrast(0.2),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.3)
  ]
)


my usual design for CNNs

In [13]:
model1 = Sequential([
    data_augmentation,
    Rescaling(1./255),
    Conv2D(16, 5, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(256, 3, activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(5)
])

In [14]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 220, 220, 16)      1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 108, 108, 64)      9280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 256)      

In [15]:
model2 = Sequential([
  data_augmentation,
  Rescaling(1./255),
  Conv2D(16, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Conv2D(32, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Conv2D(64, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Dropout(0.2),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(5)
])

In [16]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 64)       

In [17]:
model3 = Sequential([
    data_augmentation,
    Rescaling(1./255),
    Conv2D(16, 5, activation='relu'),
    MaxPooling2D(),
    Conv2D(16, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(5)
])

In [18]:
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
rescaling_2 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 220, 220, 16)      1216      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 110, 110, 16)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 108, 108, 16)      2320      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 54, 54, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 52, 52, 64)       

In [19]:
model4 = Sequential([
    data_augmentation,
    Rescaling(1./255),
    Conv2D(16, 5, activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 5, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(5)
])

In [20]:
model4.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
rescaling_3 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 220, 220, 16)      1216      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 110, 110, 16)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 106, 106, 32)      12832     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 53, 53, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 51, 51, 64)       

In [21]:
model5 = Sequential([
    Rescaling(1./255,  input_shape=(*Config.img_dim, 3)),
    Conv2D(32, 5, activation='relu',),
    MaxPooling2D(),
    Conv2D(64, 5, activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 5, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(5)
])

In [22]:
model5.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_4 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 220, 220, 32)      2432      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 106, 106, 64)      51264     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 53, 53, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 49, 49, 128)       204928    
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 24, 24, 128)      

In [23]:
model6 = Sequential([
    Rescaling(1./255,  input_shape=(*Config.img_dim, 3)),
    Conv2D(16, 7, activation='relu',),
    MaxPooling2D(),
    Conv2D(64, 5, activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(5)
])

In [24]:
model6.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_5 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 218, 218, 16)      2368      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 109, 109, 16)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 105, 105, 64)      25664     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 52, 52, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 25, 25, 128)      

In [25]:
model7 = Sequential([
    data_augmentation,
    Rescaling(1./255),
    Conv2D(16, 3, activation='relu',),
    MaxPooling2D(),
    Conv2D(32, 5, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 7, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(5)
])

In [26]:
model7.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
rescaling_6 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 107, 107, 32)      12832     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 53, 53, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 47, 47, 64)       

In [27]:
model8 = Sequential([
    Rescaling(1./255,  input_shape=(*Config.img_dim, 3)),
    Conv2D(16, 5, activation='relu',),
    MaxPooling2D(),
    Conv2D(32, 5, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(5)
])

In [28]:
model8.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_7 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 220, 220, 16)      1216      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 110, 110, 16)      0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 106, 106, 32)      12832     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 53, 53, 32)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 89888)             0         
_________________________________________________________________
dense_21 (Dense)             (None, 256)              

In [29]:
model9 = Sequential([
    Rescaling(1./255,  input_shape=(*Config.img_dim, 3)),
    Conv2D(16, 3, activation='relu',),
    MaxPooling2D(),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(5)
])

In [30]:
model9.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_8 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 52, 52, 128)       36992     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 26, 26, 128)      

In [31]:
model10 = Sequential([
    data_augmentation,
    Rescaling(1./255),
    Conv2D(16, 5, activation='relu',),
    MaxPooling2D(),
    Conv2D(32, 5, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(256, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(5)
])

In [32]:
model10.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
rescaling_9 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 220, 220, 16)      1216      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 110, 110, 16)      0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 106, 106, 32)      12832     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 53, 53, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 51, 51, 64)      

In [33]:
model11 = Sequential([
    Rescaling(1./255, input_shape=(*Config.img_dim, 3)),
    Conv2D(16, 3, activation='relu',),
    Conv2D(16, 3, activation='relu',),
    MaxPooling2D(),
    Conv2D(32, 3, activation='relu'),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, activation='relu'),
    Conv2D(128, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(256, 3, activation='relu'),
    Conv2D(256, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(32, activation='relu'),
    Dense(5)
])

In [34]:
model11.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_10 (Rescaling)     (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 222, 222, 16)      448       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 220, 220, 16)      2320      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 110, 110, 16)      0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 108, 108, 32)      4640      
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 106, 106, 32)      9248      
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 53, 53, 32)      

In [35]:
models = [model1, model2, model3, model4, model5, model6, model7, model8, model9, model10, model11]

In [36]:
model_stats = []
for i, model in enumerate(models, start=1):
    print(f"\n\n\n{'='*25} MODEL {i}/{len(models)} {'='*25}\n\n\n")
    model.compile(optimizer='adam', 
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'],
             )
    model.fit(train_ds, validation_data=val_ds, epochs=Config.epochs)
    params = model.count_params()
    loss = model.history.history['loss'][-1]
    acc = model.history.history['accuracy'][-1]
    val_acc = model.history.history['val_accuracy'][-1]
    model_stats.append((params, loss, acc, val_acc))




========================= MODEL 1/11 =========================



Epoch 1/20


2022-07-06 07:30:12.247735: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-06 07:30:14.692634: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


13/13 [==============================] - 12s 151ms/step - loss: 2.4694 - accuracy: 0.1949 - val_loss: 1.5911 - val_accuracy: 0.1875
Epoch 2/20
13/13 [==============================] - 2s 97ms/step - loss: 1.5957 - accuracy: 0.2205 - val_loss: 1.6224 - val_accuracy: 0.1875
Epoch 3/20
13/13 [==============================] - 2s 95ms/step - loss: 1.5788 - accuracy: 0.2410 - val_loss: 1.5478 - val_accuracy: 0.2708
Epoch 4/20
13/13 [==============================] - 2s 93ms/step - loss: 1.5746 - accuracy: 0.2769 - val_loss: 1.5627 - val_accuracy: 0.2083
Epoch 5/20
13/13 [==============================] - 2s 93ms/step - loss: 1.5516 - accuracy: 0.2564 - val_loss: 1.6686 - val_accuracy: 0.1875
Epoch 6/20
13/13 [==============================] - 2s 95ms/step - loss: 1.5306 - accuracy: 0.2410 - val_loss: 1.5619 - val_accuracy: 0.2500
Epoch 7/20
13/13 [==============================] - 2s 96ms/step - loss: 1.4938 - accuracy: 0.3026 - val_loss: 1.6316 - val_accuracy: 0.3333
Epoch 8/20
13/13 [====

In [37]:
for i, (params, loss, acc, val_acc) in enumerate(model_stats, start=1):
    print(f"Model {i}: {params} parameters\n")
    print(f"Loss: {loss}")
    print(f"Train Accuracy: {acc}")
    print(f"Vallidation Accuracy: {val_acc}\n {'-'*50}")

Model 1: 44497989 parameters

Loss: 1.2147085666656494
Train Accuracy: 0.4615384638309479
Vallidation Accuracy: 0.5416666865348816
 --------------------------------------------------
Model 2: 6446885 parameters

Loss: 1.231062889099121
Train Accuracy: 0.46666666865348816
Vallidation Accuracy: 0.3958333432674408
 --------------------------------------------------
Model 3: 2413493 parameters

Loss: 1.6092618703842163
Train Accuracy: 0.23076923191547394
Vallidation Accuracy: 0.1875
 --------------------------------------------------
Model 4: 10289573 parameters

Loss: 1.3380918502807617
Train Accuracy: 0.4000000059604645
Vallidation Accuracy: 0.3333333432674408
 --------------------------------------------------
Model 5: 38074181 parameters

Loss: 0.6000689268112183
Train Accuracy: 0.7846153974533081
Vallidation Accuracy: 0.4375
 --------------------------------------------------
Model 6: 10350597 parameters

Loss: 0.8069069385528564
Train Accuracy: 0.6666666865348816
Vallidation Accuracy